In [ ]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt

data = pd.read_csv('/Users/arshvetsoff/Desktop/DIPLOMA/atsenergo_2017_2023_1.csv')
data['is_mon'] = 0
data['is_sat'] = 0
data['is_sun'] = 0
data.index = data.timestamp
data.drop(columns=['timestamp'], inplace=True)
data.index = pd.to_datetime(data.index)
#start from 13-03-2020 till 12-03-2021 (365 days)
data = data.iloc[28008:28008+12264]
data = data.resample('D').mean()
data['is_mon'] = data.index.weekday == 0
data['is_sat'] = data.index.weekday == 5
data['is_sun'] = data.index.weekday == 6


In [ ]:
#plot train-test split
# TRAIN_SIZE = 7008
# TEST_SIZE = 1752

plt.figure(figsize=(10,5))
plt.plot(data['price'], label='target')
plt.xticks(rotation=60)
plt.axvline(x = data.iloc[397:397+2].index[0], color = 'g', linestyle='--')
plt.gcf().tight_layout()
plt.legend()
plt.show()

In [ ]:
# prepare data for experimentabs
# using two weeeks to predict two weeks forward
whole_train = data.iloc[:7008-1200]
whole_train.index = whole_train.index.astype('period[D]')


train_df = data.iloc[:397]
train_df.index = train_df.index.astype('period[D]')
prices_train = np.log(train_df.price.values.astype(float))
mondays_train = train_df.is_mon.values.astype(int)
saturdays_train = train_df.is_sat.values.astype(int)
sundays_train = train_df.is_sun.values.astype(int)
train_df['price_log'] = np.log(train_df['price'])

test_df = data.iloc[397:]
test_df.index = test_df.index.astype('period[D]')
prices_test = np.log(test_df.price.values.astype(float))
mondays_test = test_df.is_mon.values.astype(int)
saturdays_test = test_df.is_sat.values.astype(int)
sundays_test = test_df.is_sun.values.astype(int)

len(prices_train), len(prices_test)

In [ ]:
train_data = {
    'N' : len(prices_train),
    'y' : list(prices_train),
    'D_Sat' : list(saturdays_train),
    'D_Sun' : list(sundays_train),
    'D_Mon' : list(mondays_train)
}

test_data = []
for i in range(len(prices_test)):
    test_data.append({
        'y' : prices_test[i],
        'D_Sat' : saturdays_test[i],
        'D_Sun' : sundays_test[i],
        'D_Mon' : mondays_test[i]
    })

In [ ]:
variance = []
cur_variance = 0.8
while cur_variance < 1.0:
    variance.append(cur_variance)
    cur_variance += 0.025

index = []
cur_index = 4.5
while cur_index < 6.1:
    index.append(cur_index)
    cur_index += 0.5

In [ ]:
from cmdstanpy import CmdStanModel
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, root_mean_squared_error


for cur_var in variance:
    for cur_ind in index:
        model_code = """
        data {{
            int<lower=0> N;  // number of time points
            vector[N] y;     // log prices
            vector[N] D_Sat;  // Indicator for Saturday
            vector[N] D_Sun;  // Indicator for Sunday
            vector[N] D_Mon;  // Indicator for Monday
        }}
        parameters {{
            real mu;
            real<lower=0, upper=2> kappa_h;
            real theta_h;
            real d_sat;
            real d_sun;
            real d_mon;
            real<lower=0, upper=1> rho_raw;
            vector[N] eps1;
            vector[N] eps2;
            simplex[3] p; // Dirichlet distribution for jumps
            real n_U;
            real n_D;
            vector<lower=0.1>[N] xi_d;
            vector<lower=0.1>[N] xi_u;
        }}
        transformed parameters {{
            vector[N] h; //parameter of model
            vector[N] j; //parameter of model

            real rho = 2 * rho_raw - 1; // (-1, 1)
            real sigma_h = 0.01;

            h[1] = 0;
            for (i in 2:N) {{
                h[i] = h[i-1] + kappa_h * (theta_h - h[i-1]) + sigma_h * (rho * eps1[i] + sqrt(1 - rho^2) * eps2[i]);
            }}
        }}
        model {{
            // Y VARS
            mu ~ normal(0, {cur_v});
            d_sat ~ normal(0, {cur_v});
            d_sun ~ normal(0, {cur_v});
            d_mon ~ normal(0, {cur_v});
            // H VARS
            kappa_h ~ normal(1, {cur_i});
            theta_h ~ normal(0, {cur_v});
            eps1 ~ normal(0, 1);
            eps2 ~ normal(0, 1);
            rho_raw ~ beta(5, 5);
            // JUMP VARS
            p ~ dirichlet([1,1,1]);
            n_U ~ inv_gamma(1.86, 0.43);
            n_D ~ inv_gamma(1.86, 0.43);
            xi_d ~ exponential(n_D);
            xi_u ~ exponential(n_U);
        }}
        generated quantities {{
            vector[N] y_pred_train;
            vector[N] jumps;
            vector[N] q;

            {{
                for (i in 1:N) {{
                q[i] = categorical_rng(p) - 2;
                }}
            }}

            {{
                for (i in 1:N) {{
                if (q[i] == -1)
                    jumps[i] = -1 * xi_d[i];
                else if (q[i] == 0)
                    jumps[i] = 0;
                else
                    jumps[i] = xi_u[i];
                }}
            }}

            {{
                y_pred_train[1] = y[1];
                for (i in 2:N) {{
                    y_pred_train[i] = y[i-1] + mu + d_sat * D_Sat[i] + d_sun * D_Sun[i] + d_mon * D_Mon[i] + sqrt(exp(h[i-1])) * eps1[i] + jumps[i];
                }}
            }}
        }}
        """.format(cur_v=cur_var, cur_i=cur_ind)

        with open(f'/Users/arshvetsoff/Desktop/DIPLOMA/FUCKING_STAN_PREDICTIONS/models/parameters_jump_baseline/jump_{cur_ind}_{cur_var}.stan', 'a') as f:
            f.write(model_code)

        stan_model = CmdStanModel(stan_file=f'/Users/arshvetsoff/Desktop/DIPLOMA/FUCKING_STAN_PREDICTIONS/models/parameters_jump_baseline/jump_{cur_ind}_{cur_var}.stan')
        fit = stan_model.sample(data=train_data, iter_sampling=50000, iter_warmup=30000, chains=1, adapt_delta=0.95)
        check_df = pd.DataFrame({'pred' : np.quantile(fit.stan_variable('y_pred_train'), q=0.5, axis=0)}, index = train_df.index.values[:397])
        print("#################################")
        print(f"cur variance is {cur_var}, cur index is {cur_ind}")
        print('MSE: ', mean_squared_error(np.exp(train_df['price_log']), np.exp(check_df['pred'])))
        print('RMSE: ', root_mean_squared_error(np.exp(train_df['price_log']), np.exp(check_df['pred'])))
        print('MAPE: ', mean_absolute_percentage_error(np.exp(train_df['price_log']), np.exp(check_df['pred'])))   
        print("#################################")